# AS - TME7
2018--2019

---

Binome : Juliette Ortholand, Stieban Fernandez

Modèles de séquence (réccurents)
- Modèle sur les caractères (prédire classe)
- Modèle génératif

Données Page AS@DAC
-> names (classes) :
- english.txt
- french.txt
- italian.txt
- ...

Chaque caractère en 1-hot -> char-embedding

Etat précédent : s0 s1, s2, ... application d'une fonction f

+

Classifieur

On travaille avec des batchs

-> remplir caractère vide par 0 par ex

---

Pour les modèles génératifs : longueur du prénom + 1 caractère pour déterminer la fin de la chaine :

Appliquer un masque : 1 s'il y a un caractère (+ caractère de fin) sinon 0

-> Prédire à chaque fois l'état (/caractère) suivant.

Utilisation : 
- Est-ce que "Pierre" est un prénom anglais ? (Calcul de la vraisemblance)
- Générer des prénoms

---

Parcourir les fichiers du répértoire avec glob('*.txt')

## Préparation des données

In [1]:
import torch
import torch.nn as nn
from unidecode import unidecode

from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt

In [2]:
ord(' ')

32

In [5]:
from pathlib import Path

#-------------------------------------------------------
# Extraction des noms et labels
#-------------------------------------------------------
gen = Path("data/names").glob("*.txt")

L = [] # [(nom, pays), ...]
D = {} # {label(int): pays, ...}
for label, file in enumerate(gen):
    D[label] = file.stem
    
    with open(str(file), "r", encoding="utf-8") as f:
        names = f.read().split("\n")[:-1]
    for name in names:
        # Nettoyage
        name = "".join([c for c in unidecode(name.lower()) if c.isalpha()])
        L.append((name, label))
L = L[:500]
#-------------------------------------------------------
# Encodage
#-------------------------------------------------------
N = 27 # nombre de caractères différents pour le one hot

code_char = lambda x: ord(x)-96

decode_char = lambda x: chr(x+96)

def encode_name(name, N, M=None):
    # N : taille d'un vecteur one hot
    # M : padding
    res = torch.zeros((M if M!=None and M>len(name) else len(name)))
    #res = torch.zeros((M if M!=None and M>len(name) else len(name)), N)
    for i, c in enumerate(name):
        res[i] = code_char(c)
        #res[i, code_char(c)] = 1
    res[len(name):] = 1
    #res[len(name):, 0] = 1
    return res

def decode_name(indxs):
    return "".join([decode_char(indx) for indx in indxs if indx!=1])
    #return "".join([decode_char(indx.argmax()) for indx in indxs if indx[0]!=1])

#-------------------------------------------------------
# Construction du DataSet 
#-------------------------------------------------------
class NameDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, index):
        name, label = self.data[index]
        return encode_name(name, N), int(label)
    
    def __len__(self):
        return len(self.data)
    
#-------------------------------------------------------
# DataLoader
#-------------------------------------------------------

def collate_fn(sequences):
    length = max([len(x[0]) for x in sequences])
    data = torch.zeros(len(sequences), length)
    #data = torch.zeros(len(sequences), length, N)
    target = torch.zeros(len(sequences))
    for i, (name, label) in enumerate(sequences):
        data[i, :len(name)] = name
        #data[i, len(name):, 0] = 1 # padding pour s'adapter aux dimensions du batch
        data[i, len(name):] = 1
        target[i] = label
    return data, target
        
    
def get_data_loader(dataset, batch_size):
    return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, \
                         collate_fn=collate_fn)

#-------------------------------------------------------
# Split Train Val
#-------------------------------------------------------

import numpy as np
from sklearn.model_selection import train_test_split

X, y = np.array(L).T
Xtrain, Xval, ytrain, yval = train_test_split(X, y)

train_set = NameDataset(np.vstack((Xtrain, ytrain)).T)
val_set = NameDataset(np.vstack((Xval, yval)).T)

train_iter = get_data_loader(train_set, 150)
val_iter = get_data_loader(val_set, 150)

In [6]:
encode_name("pierre", N, 19)

tensor([16.,  9.,  5., 18., 18.,  5.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.])

In [85]:
for x, y in train_iter:
    ex, lab = x, y
    print(x.shape, y.shape)

torch.Size([150, 8]) torch.Size([150])
torch.Size([150, 8]) torch.Size([150])
torch.Size([75, 8]) torch.Size([75])


---

## Construction du modèle

In [71]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        
        self.emb = nn.Embedding(28,input_size)
        
        self.linx = nn.Linear(input_size, hidden_size)
        self.linh = nn.Linear(hidden_size, hidden_size)
        self.activ = nn.ReLU()
        self.liny = nn.Linear(hidden_size, 28)
        
    def forward(self, seq):
        # seq : (seq_length, batch_size)
        
        emb_seq = self.emb(seq)
        output = self.initHidden()
        
        for _input in emb_seq:
            output = self.activ(self.linx(_input) + self.linh(output))
        output = self.liny(output)
        
        return output

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

input_size, hidden_size, output_size = 50, 110, len(D)
model = RNN(input_size, hidden_size, output_size)

In [72]:
# TEST

out = model(ex.transpose(0,1).long())

for i, e in enumerate(out):
    print(decode_name(ex[i]), ":", D[e.argmax().item()])

NameError: name 'ex' is not defined

In [73]:
x.transpose(0,1).long().shape

NameError: name 'x' is not defined

---

## Classification

In [151]:
class LSTM(nn.Module):
    
    def __init__(self,inputsize,hidden_size,output_size):
        super(LSTM, self).__init__()
        
        #Couches
        self.linF = [nn.Linear(inputsize, hidden_size),nn.Linear(hidden_size, hidden_size)]
        self.linI = [nn.Linear(inputsize, hidden_size),nn.Linear(hidden_size, hidden_size)]
        self.linC = [nn.Linear(inputsize, hidden_size),nn.Linear(hidden_size, hidden_size)]
        self.linO = [nn.Linear(inputsize, hidden_size),nn.Linear(hidden_size, hidden_size)]
        self.linOp = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
        
        self.clas = nn.Linear(hidden_size, output_size)
        #Variables
        self.hidden_size = hidden_size
    
    def forward(self, X):
        c, h = self.initHidden()
        for _input in X:            
            #Bloc 1
            tmp2 = self.sigmoid(self.linF[1](h)+self.linF[0](_input))
            
            #Bloc 2
            tmp3 = self.sigmoid(self.linI[1](h)+self.linI[0](_input)) \
                    * self.tanh(self.linC[1](h)+self.linC[0](_input))
            
            #Application
            c = (c * tmp2)+ tmp3
            
            #Bloc 3 
            h = self.sigmoid(self.linO[1](h)+self.linO[0](_input)) * self.tanh(self.linOp(c))
        
        out = self.clas(h)                                     
                                                          
        return out
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size),torch.zeros(1, self.hidden_size)
    
    
    def score(self,ypred,y):
        score = 0
        if len(y) != len(ypred):
            print('ERREUR')
        for i in range(0,len(y)):
            if y[i] == ypred[i].argmax():
                score += 1
        return(score/len(y))

In [152]:
model = LSTM(input_size, hidden_size, output_size)
loss = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 2

def accuracy(ypred, y):
    ### Calcul du score
    return (ypred.argmax(dim=1) == y).float().mean()

def fit_eval_rnn(model, loss, optim, train_iter, val_iter, n_epochs):
    train_loss = []
    train_score = []
    val_loss = []
    val_score = []

    ### APPRENTISSAGE
    #--------------------------------------------------------
    for i in range(n_epochs):
        
        model.train()
        ####### TRAIN #######
        costs_train = []
        scores_train = []
        for x, l in train_iter:
            x = x.transpose(0,1).long()
            for i in range(0,len(x)-1):
                x_seq = x[:i+1]
                y_seq = x[i:i+1].view(-1)

                out = model(x_seq)
                cost = loss(out, y_seq)
                optim.zero_grad()
                cost.backward()
                optim.step()

                costs_train.append(cost.float())
                scores_train.append(float(accuracy(out, y_seq)))

            train_loss.append(torch.tensor(costs_train).float().mean())
            train_score.append(torch.tensor(scores_train).float().mean())

        model.eval()
        ####### VALID #######
        costs_val = []
        scores_val = []
        for x, l in val_iter:
            x = x.transpose(0,1).long()
            for i in range(0,len(x)-1):
                x_seq = x[:i+1]
                y_seq = x[i:i+1].view(-1)
                
                out = model(x_seq)
                cost = loss(out, y_seq)

                costs_val.append(cost.float())
                scores_val.append(float(accuracy(out, y_seq)))

            val_loss.append(torch.tensor(costs_val).float().mean())
            val_score.append(torch.tensor(scores_val).mean())
        
    return train_loss, val_loss, train_score, val_score


def show_loss_score(train_loss, val_loss, train_score, val_score):
    ### AFFICHAGE LOSS ET SCORE
    #--------------------------------------------------------
    plt.figure(figsize=(10,7))
    plt.plot(train_loss, c="red", label="train")
    plt.plot(val_loss, c="b", label="val")
    plt.legend(loc="best")
    plt.title('Loss')
    plt.show()

    plt.figure(figsize=(10,7))
    plt.plot(train_score, c="red", label="train")
    plt.plot(val_score, c="b", label="val")
    plt.legend(loc="best")
    plt.title('score')
    plt.show()

In [153]:
res = fit_eval_rnn(model, loss, optim, train_iter, val_iter, n_epochs)
show_loss_score(*res)

RuntimeError: Expected object of type torch.LongTensor but found type torch.FloatTensor for argument #2 'mat2'

In [62]:
import collections

collections.Counter(list(ytrain)+list(yval))

Counter({'0': 500})

Résultat biaisé car certaines classes contiennent plus d'exemples que d'autres

--> Preprocessing nécessaire : rebalancage des classes pour l'apprentissage

## Génération

In [137]:
test = encode_name("robbin", N, 19)
decode_name(test[:4])

'robb'

In [138]:
res = model(test[:4].long())

In [139]:
decode_name(res.argmax().view(1))

'b'